# Capstone Project - The Battle of the Neighborhoods

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
1. Number of existing restaurants in the neighborhood (any type of restaurant)
1. Number of and distance to Chinese restaurants in the neighborhood, if any
distance of neighborhood from city center.

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.
Following data sources will be needed to extract/generate the required information:
centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using LocationIQ API reverse geocoding
number of restaurants and their type and location in every neighborhood will be obtained using Foursquare API;
coordinate of The Bronx center will be obtained using LocationIQ API geocoding of a well known location (NY Botanical Garden)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around NY Botanical Garden.

Let's first find the latitude & longitude of The Bronx center, using specific, well known address and LocationIQ geocoding API.

In [25]:
# The code was removed by Watson Studio for sharing.

In [2]:
import requests
import json

def get_coordinates(key, q, verbose=False):
    try:
        url = 'https://us1.locationiq.com/v1/search.php?key={}&q={}&format=json'.format(key, q)
        response = requests.get(url).json()
        if verbose:
            print('API JSON result =>', response)
        results = json.dumps(response)
        boundingbox = response[0]['boundingbox'] # get geographical coordinates
        lat = response[0]['lat']
        lon = response[0]['lon']
        return [lat, lon]
    except:
        return [None, None]

In [3]:
address = 'The Bronx, New York, United States'
ny_center = get_coordinates(key, q)
print('Coordinate of {}: {}'.format(address, ny_center))

Coordinate of The Bronx, New York, United States: ['40.86224305', '-73.8771540952978']


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from NY Botanical Garden. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.
To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).

In [4]:
!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('NY center longitude = {}, latitude = {}'.format(ny_center[1], ny_center[0]))
x, y = lonlat_to_xy(ny_center[1], ny_center[0])
print('NY center UTM X = {}, Y = {}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('NY center longitude = {}, latitude = {}'.format(lo, la))

Coordinate transformation check
-------------------------------
NY center longitude = -73.8771540952978, latitude = 40.86224305
NY center UTM X = -5796498.221803047, Y = 9853727.256589096
NY center longitude = -73.87715409529737, latitude = 40.86224304999892


Let's create a hexagonal grid of cells: we offset every other row, and adjust vertical row spacing so that every cell center is equally distant from all it's neighbors.

In [5]:
ny_center_x, ny_center_y = lonlat_to_xy(ny_center[1], ny_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = ny_center_x - 6000
x_step = 600
y_min = ny_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(ny_center_x, ny_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [6]:
!pip install folium

import folium

In [26]:
map_ny = folium.Map(location = ny_center, zoom_start = 13)
folium.Marker(ny_center, popup = 'New York Botanical Garden').add_to(map_ny)
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=300, color = 'red', fill = False).add_to(map_ny)
map_ny

In [24]:
# The code was removed by Watson Studio for sharing.

In [9]:
def get_address(key, lat_x, lon_x, verbose=False):
    try:
        url = 'https://us1.locationiq.com/v1/reverse.php?key={}&lat={}&lon={}&format=json'.format(key, lat_x, lon_x)
        response = requests.request('GET', url=url)
        if verbose:
            print('API JSON result =>', response)
        results = response.json()
        address = results['display_name']
        return address
    except:
        return None

addr = get_address(key, ny_center[0], ny_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(ny_center[0], ny_center[1], addr))

Reverse geocoding check
-----------------------
Address of [40.86224305, -73.8771540952978] is: New York Botanical Garden, Bronx Park, New York, Bronx County, New York, USA


In [10]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', US', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [11]:
addresses[1:50]

['2853, East 196th Street, Pelham Bay, New York, Bronx County, New York, 10461A',
 '1826, Stillwell Ave, Pelham Gardens, New York, Bronx County, New York, 10469A',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'Hutchinson River Parkway, Pelham Bay, New York, Bronx County, New York, 10461A',
 '1200, Waters Pl, Pelham Bay, New York, Bronx County, New York, 10461A',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 '2917, Ely Ave, Baychester, New York, Bronx County, New York, 10469A',
 '1947, Hammersley Avenue, Baychester, New York, Bronx County, New York, 10469A',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 '1575, Pelham Pkwy N, Pelham Gardens, New York, Bronx County, New York, 10461A',
 '2414, Mickle Ave, Pelham Gardens, New York, Bronx County, New York, 10469A',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 '3234, Wickham Avenue, Baychester, New York, Bronx County, New York, 10469A',
 '3415, Grace A

In [12]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"1941, Pilgrim Ave, Pelham Bay, New York, Bronx...",40.850584,-73.833113,-5.798298e+06,9.848011e+06,5992.495307
1,"2853, East 196th Street, Pelham Bay, New York,...",40.854128,-73.832968,-5.797698e+06,9.848011e+06,5840.376700
2,"1826, Stillwell Ave, Pelham Gardens, New York,...",40.857672,-73.832823,-5.797098e+06,9.848011e+06,5747.173218
3,NO ADDRESS,40.861217,-73.832679,-5.796498e+06,9.848011e+06,5715.767665
4,NO ADDRESS,40.864761,-73.832534,-5.795898e+06,9.848011e+06,5747.173218
5,NO ADDRESS,40.868306,-73.832389,-5.795298e+06,9.848011e+06,5840.376700
6,NO ADDRESS,40.871851,-73.832245,-5.794698e+06,9.848011e+06,5992.495307
7,NO ADDRESS,40.845364,-73.837370,-5.799198e+06,9.848531e+06,5855.766389
8,"Hutchinson River Parkway, Pelham Bay, New York...",40.848908,-73.837226,-5.798598e+06,9.848531e+06,5604.462508
9,"1200, Waters Pl, Pelham Bay, New York, Bronx C...",40.852451,-73.837082,-5.797998e+06,9.848531e+06,5408.326913


In [13]:
df_locations.to_pickle('./locations.pkl')    

Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.
We're interested in venues in 'food' category, so we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Chinese restaurant' category, as we need info on Chinese restaurants in the neighborhood.

In [14]:
# @hiddencell
client_id = 'I1QQKNVV4PQXBZXEU2CHBLZIMLD4LBTBODR12CEGF23ASPMY'
client_secret = 'FORSGYRRFIEKBNRRPJ0YT4BIQ14BOEZ2222U52P0KGYKFWET'

In [15]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

chinese_restaurant_categories = ['4bf58dd8d48988d145941735','52af3a5e3cf9994f4e043bea','52af3a723cf9994f4e043bec',
                                 '52af3a7c3cf9994f4e043bed','58daa1558bbb0b01f18ec1d3','52af3a673cf9994f4e043beb',
                                 '52af3a903cf9994f4e043bee','4bf58dd8d48988d1f5931735','52af3a9f3cf9994f4e043bef',
                                 '52af3aaa3cf9994f4e043bf0','52af3ab53cf9994f4e043bf1','52af3abe3cf9994f4e043bf2',
                                 '52af3ac83cf9994f4e043bf3','52af3ad23cf9994f4e043bf4','52af3add3cf9994f4e043bf5',
                                 '52af3af23cf9994f4e043bf7','52af3ae63cf9994f4e043bf6','52af3afc3cf9994f4e043bf8',
                                 '52af3b053cf9994f4e043bf9','52af3b213cf9994f4e043bfa','52af3b293cf9994f4e043bfb',
                                 '52af3b343cf9994f4e043bfc','52af3b3b3cf9994f4e043bfd','52af3b463cf9994f4e043bfe',
                                 '52af3b633cf9994f4e043c01','52af3b513cf9994f4e043bff','52af3b593cf9994f4e043c00',
                                 '52af3b6e3cf9994f4e043c02','52af3b773cf9994f4e043c03','52af3b813cf9994f4e043c04',
                                 '52af3b893cf9994f4e043c05','52af3b913cf9994f4e043c06','52af3b9a3cf9994f4e043c07',
                                 '52af3ba23cf9994f4e043c08']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Chinese', '')
    address = address.replace(', China', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20200314'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [16]:
# Let's now go over our neighborhood locations and get nearby restaurants; 
#we'll also maintain a dictionary of all found restaurants and all found chinese restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    chinese_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage 
        #so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, client_id, client_secret, radius=350, 
                                          limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_chinese = is_restaurant(venue_categories, specific_filter = chinese_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], 
                              venue_address, venue_distance, is_chinese, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_chinese:
                    chinese_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, chinese_restaurants, location_restaurants

# Try to load from local file system, in case we did this before
restaurants = {}
chinese_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('chinese_restaurants_350.pkl', 'rb') as f:
        chinese_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed, use the Foursquare API to get the data
if not loaded:
    restaurants, chinese_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('chinese_restaurants_350.pkl', 'wb') as f:
        pickle.dump(chinese_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Restaurant data loaded.


In [17]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Chinese restaurants:', len(chinese_restaurants))
print('Percentage of Chinese restaurants: {:.2f}%'.format(len(chinese_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 614
Total number of Chinese restaurants: 148
Percentage of Chinese restaurants: 24.10%
Average number of restaurants in neighborhood: 2.9175824175824174


In [18]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4b85b6b3f964a5209b6e31e3', "Giovanni's Italian Restaurant", 40.85178560311938, -73.82978599125016, '3209 Westchester Ave, Bronx, NY 10461, United States', 310, False, -5798081.787721744, 9847590.526907336)
('4c9e1ef2542b224b70a3f09f', 'Lucca Italian Restaurant', 40.847676, -73.831834, '3019 Westchester Ave (Hobart Ave), Bronx, NY 10461, United States', 341, False, -5798785.111633937, 9847832.005488586)
('4b4c90d3f964a52046b626e3', "Honey's Thai Pavilion", 40.84784479852375, -73.83131988215104, '3036 Westchester Ave (At Hobart Ave), Bronx, NY 10461, United States', 340, False, -5798754.508530283, 9847766.850168414)
('4b5f3d97f964a52074af29e3', 'Lee Xing', 40.85164023136602, -73.82979925838885, '3207 Westchester Ave, Bronx, NY 10461, United States', 302, True, -5798106.429940105, 9847591.46690517)
('4e4dd9bcbd4101d0d79cbea8', 'Good Friend Chinese Kitchen', 40.853797, -73.830832, '2099 Hobart Ave, Bronx, NY 10461, United States', 183, True

In [19]:
print('List of Chinese restaurants')
print('---------------------------')
for r in list(chinese_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(chinese_restaurants))

List of Chinese restaurants
---------------------------
('4b5f3d97f964a52074af29e3', 'Lee Xing', 40.85164023136602, -73.82979925838885, '3207 Westchester Ave, Bronx, NY 10461, United States', 302, True, -5798106.429940105, 9847591.46690517)
('4e4dd9bcbd4101d0d79cbea8', 'Good Friend Chinese Kitchen', 40.853797, -73.830832, '2099 Hobart Ave, Bronx, NY 10461, United States', 183, True, -5797745.744014473, 9847735.434333028)
('4d865bbaf1e56ea8cd5d9c8a', 'Eastern Wok Buffet', 40.863575, -73.83448, 'New York, United States', 303, True, -5796106.557223862, 9848255.110053103)
('4d42202033268cfaafb65501', 'Pearl Of China', 40.843238, -73.83551999999999, '2830 Middletown Rd, Bronx, NY 10461, United States', 283, True, -5799550.550375677, 9848282.279303517)
('5036bfcfe4b0cc82fc5cc1f2', 'Joyful Dragon', 40.86895881105623, -73.83417857024543, '2829 Edson Ave, Bronx, NY 10469, United States', 228, True, -5795194.957444468, 9848244.583470743)
('4eadecbf9911d5c95181ee84', 'Yummy House', 40.86249, -73.

In [20]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 130):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: 900 Park Restaurant, Maestro's Caterers, Lotus Flame Restaurant, Mambo Italiano
Restaurants around location 102: F & J Pine Tavern
Restaurants around location 103: Azul Tequila, Panda restaurant
Restaurants around location 104: 
Restaurants around location 105: 
Restaurants around location 106: 
Restaurants around location 107: 
Restaurants around location 108: Lee's Chinese Kitchen Of The Bronx
Restaurants around location 109: Crown Fried Chicken, Spices
Restaurants around location 110: Top Of The Hill Restaurant
Restaurants around location 111: Roc A Tone Restaurant, Chai Hwa Chinese Restaurant
Restaurants around location 112: Gold Star Jerk And Seafood Center, Golden Krust Caribbean Restaurant, Gooddine, Gold Star, Agra Grill, Parrilla Internacional, EC Soul Food And Seafood, Chai Hwa Chinese Restaurant
Restaurants around location 113: Bx Uptown Caribbean Taste, Gooddine, Mango Caribbean Restaur

In [21]:
map_ny = folium.Map(location = ny_center, zoom_start=13)
folium.Marker(ny_center, popup = 'New York Botanical Garden').add_to(map_ny)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_chinese = res[6]
    color = 'red' if is_chinese else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_ny)
map_ny

Looking good. So now we have all the restaurants in area within few kilometers from New York Botanical Garden, and we know which ones are Chinese restaurants! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.
This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Chinese restaurant!